In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from collections import defaultdict # For easier subject counts
import operator
from tqdm.notebook import tqdm
import datetime
import re
import pickle
import random
# Optional: for better display in Jupyter
from IPython.display import display
from scipy.stats import truncnorm

In [3]:
df = pd.read_csv('test_network_base.csv')
print("Loaded ColocCotime_network_base.csv with shape:", df.shape)
df.head()

Loaded ColocCotime_network_base.csv with shape: (11411, 690)


,College,Subject,Year,Social Propensity,Club Propensity,Disease State,Adapted Group Size,Gender,Assigned,Group ID,...,T_668,T_669,T_670,T_671,StudyBudget,SocialBudget,OutdoorBudget,WeeklySocialBudgets,WeeklySocialBlocks,SleepRequirement
0,Christ's,Architecture Tripos,1,10,7,0,7,Female,True,1.0,...,Other,Sainsbury's (Sidney Street),Other,Other,148,98,16,"[32.666666666666664, 21.777777777777775, 21.77...","[[6, 4, 6, 5, 2, 6, 5, 5, 4], [3, 3, 5, 5, 4, ...",6
1,Christ's,Architecture Tripos,1,10,10,0,6,Female,True,2.0,...,Other,Other,Other,Other,142,103,4,"[34.33333333333333, 22.88888888888889, 22.8888...","[[4, 3, 5, 5, 4, 6, 6, 5, 4, 3], [4, 3, 3, 3, ...",6
2,Christ's,Architecture Tripos,2,8,6,0,5,Female,True,3.0,...,Christ's Library,Christ's Library,Christ's Library,Room Posts,158,67,16,"[22.333333333333332, 14.888888888888888, 14.88...","[[5, 5, 4, 5, 6, 5, 3], [5, 3, 4, 4, 5, 4], [5...",6
3,Christ's,Architecture Tripos,2,9,9,0,6,Female,True,32.0,...,Christ's College bar,Christ's College bar,Christ's College bar,Christ's College bar,158,72,16,"[24.0, 16.0, 16.0, 16.0]","[[4, 4, 5, 5, 4, 4, 4, 3, 1], [3, 5, 5, 6, 3, ...",8
4,Christ's,Architecture Tripos,3,3,4,0,7,Male,True,5.0,...,Own Room,Other,Other,Other,167,58,48,"[19.333333333333332, 12.888888888888888, 12.88...","[[5, 4, 6, 6, 5, 4], [5, 6, 3, 2, 5, 2], [4, 4...",6


So now we have a data frame where we can infect an initial set of students. And watch the disease spread. So the method for this is. At T = 12 (skips teething problems of calender), infect 1% of students with one of 5 diseases A, B, C, D or E. If a student is not infected with a disease, their disease state should be 0. If they are infected with a disease, it should be the capital letter of the disease. Once the student is recovered, their disease state should be the lower case equivalent letter. I'm planning on using a SIR model for each timestep, where the local connection are the students that are co-spatial at that shared time. Begin by producing a Disease class with parameters: transmission rate (between 0 and 1), and recovery rate (likelihood of healing in that hour). Loading Location_popularity.csv provides every location with the number of people you a student comes in contact with while at that location in a given hour. Code up this disease class, and provide a SIR model framework -> moving from time step to time step, updating the assoicated disease column with the correct code. 



In [4]:
class Disease:
    def __init__(self, name, transmission_rate, recovery_rate):
        self.name = name  # e.g. 'A'
        self.transmission_rate = transmission_rate  # probability of infection per contact per hour
        self.recovery_rate = recovery_rate  # probability of recovery per hour

# Example: create 5 diseases
diseases = [
    Disease('A', transmission_rate=0.1, recovery_rate=0.01), #Students ill for on average, 5 days. 
    Disease('B', transmission_rate=0.05, recovery_rate=0.01),
    Disease('C', transmission_rate=0.02, recovery_rate=0.01),
    Disease('D', transmission_rate=0.015, recovery_rate=0.01),
    Disease('E', transmission_rate=0.01, recovery_rate=0.01),
    Disease('F', transmission_rate=0.005, recovery_rate=0.01),
]

In [5]:
# Load the base DataFrame
df = pd.read_csv('ColocCotime_network_base.csv')
# --- Load Location Popularity ---
# Assumes Location_popularity.csv has columns: Location, Hour, Popularity
loc_pop = pd.read_csv('Location_popularity.csv')
def get_contacts(loc, hour_of_day):
    try:
        return loc_pop.loc[loc_pop['Hour'] == hour_of_day, loc].values[0]
    except Exception:
        return 10  # fallback if missing
    
# Identify all disease state columns (or create them if not present)
time_cols = [col for col in df.columns if col.startswith('T_')]
for col in time_cols:
    disease_col = f'Disease_{col}'
    df[disease_col] = '0' * len(diseases)     # Set all disease state columns to '00000' (all susceptible/never infected)

# Save the initialized DataFrame
df.to_csv('Disease_track.csv', index=False)
print("Initialized Disease_track.csv: all disease state columns set to '00000'")


C:\Users\joshu\AppData\Local\Temp\ipykernel_28820\218508725.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[disease_col] = '0' * len(diseases)     # Set all disease state columns to '00000' (all susceptible/never infected)
C:\Users\joshu\AppData\Local\Temp\ipykernel_28820\218508725.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[disease_col] = '0' * len(diseases)     # Set all disease state columns to '00000' (all susceptible/never infected)
C:\Users\joshu\AppData\Local\Temp\ipykernel_28820\218508725.py:16: Perfo

Initialized Disease_track.csv: all disease state columns set to '00000'


In [6]:
df = pd.read_csv('Disease_track.csv')
n_students = len(df)
np.random.seed(42)
initial_indices = np.random.choice(df.index, 20, replace=False)

# Set all students to '00000' (never had a disease) at T=12
#df['Disease_T_12'] = '00000'
df['Disease_T_12'] = '0' * len(diseases)
# Infect the 20 students with all diseases (state '11111')
for idx in initial_indices:
    df.at[idx, 'Disease_T_12'] = '1' * len(diseases)

print(f"Infected {len(initial_indices)} students at T=12 with state '11111' (all diseases active)")

Infected 20 students at T=12 with state '11111' (all diseases active)


In [7]:

def _convert_state_to_matrices(disease_series, num_diseases):
    """(Helper) Converts the '012...' string column to S, I, R boolean matrices."""
    state_char_matrix = np.array([list(s) for s in disease_series.astype(str).str.zfill(num_diseases)])
    S = (state_char_matrix == '0')
    I = (state_char_matrix == '1')
    R = (state_char_matrix == '2')
    return S, I, R

def _convert_matrices_to_state(S, I, R):
    """(Helper) Converts S, I, R boolean matrices back to a '012...' string series."""
    final_state_chars = np.empty(S.shape, dtype='<U1')
    final_state_chars[S] = '0'
    final_state_chars[I] = '1'
    final_state_chars[R] = '2'
    return ["".join(row) for row in final_state_chars]


def run_sir_step_by_location(df, diseases, t, loc_pop, infection_log=None):
    """
    Processes an entire timestep using a vectorized, probabilistic, matrix-based approach.
    Additionally, logs each infection event (who infected whom, where, and when) to a DataFrame.
    If infection_log is provided, appends new infection events to it.
    Returns updated df and infection_log.
    """
    num_students = len(df)
    num_diseases = len(diseases)
    time_col = f'T_{t}'
    disease_col = f'Disease_T_{t}'
    next_disease_col = f'Disease_T_{t+1}'

    S, I, R = _convert_state_to_matrices(df[disease_col], num_diseases)
    S_next, I_next, R_next = S.copy(), I.copy(), R.copy()

    # --- Location and population matrices ---
    student_locs = df[time_col].to_numpy()
    unique_locs, loc_indices = np.unique(student_locs[pd.notna(student_locs)], return_inverse=True)
    notna_mask = pd.notna(df[time_col])
    L = np.zeros((num_students, len(unique_locs)), dtype=bool)
    L[notna_mask.to_numpy().nonzero()[0], loc_indices] = True
    pop_per_loc = L.sum(axis=0)
    hour_of_day = t % 24
    loc_pop_hour = loc_pop[loc_pop['Hour'] == hour_of_day].squeeze()
    DEFAULT_FALLBACK_RATE = 5
    contact_rates = loc_pop_hour.reindex(unique_locs, fill_value=DEFAULT_FALLBACK_RATE).values

    # --- Infection logging ---
    infection_events = []

    group_ids = df['Group ID'].values if 'Group ID' in df.columns else np.array([None]*num_students)

    for d, disease in enumerate(diseases):
        S_d, I_d = S[:, d], I[:, d]

        # --- Special handling for Room Afters ---
        for loc_idx, loc in enumerate(unique_locs):
            students_in_loc = np.where(L[:, loc_idx])[0]
            if loc == "Room Afters":
                # Only allow infection within groups
                # Group students by group id
                group_to_students = {}
                for idx in students_in_loc:
                    gid = group_ids[idx]
                    if pd.isna(gid):
                        continue
                    group_to_students.setdefault(gid, []).append(idx)
                for group, group_indices in group_to_students.items():
                    group_S = S_d[group_indices]
                    group_I = I_d[group_indices]
                    n_infectious = np.sum(group_I)
                    n_susceptible = np.sum(group_S)
                    if n_infectious == 0 or n_susceptible == 0:
                        continue
                    # Use default contact rate for Room Afters
                    inf_density = n_infectious / max(len(group_indices)-1, 1)
                    prob_no_inf = (1 - (inf_density * disease.transmission_rate)) ** DEFAULT_FALLBACK_RATE
                    prob_inf = 1 - prob_no_inf
                    rand_vals = np.random.rand(len(group_indices))
                    newly_infected = group_S & (rand_vals < prob_inf)
                    # Log infection events
                    infectious_indices = np.array(group_indices)[group_I]
                    susceptible_indices = np.array(group_indices)[newly_infected]
                    for target in susceptible_indices:
                        if len(infectious_indices) > 0:
                            source = np.random.choice(infectious_indices)
                        else:
                            source = None
                        infection_events.append({
                            'time': t,
                            'disease': disease.name,
                            'source_student': source,
                            'target_student': target,
                            'location': loc
                        })
                    # Update S/I for these group members
                    S_next[group_indices, d] = S_d[group_indices] & ~newly_infected
                    I_next[group_indices, d] = (I_d[group_indices] | newly_infected)
                # Recovery for all in Room Afters
                rand_vals_recovery = np.random.rand(len(students_in_loc))
                newly_recovered = I_d[students_in_loc] & (rand_vals_recovery < disease.recovery_rate)
                R_next[students_in_loc, d] = R[students_in_loc, d] | newly_recovered
                I_next[students_in_loc, d] = I_next[students_in_loc, d] & ~newly_recovered
            else:
                # --- Standard logic for other locations ---
                group_S = S_d[students_in_loc]
                group_I = I_d[students_in_loc]
                n_infectious = np.sum(group_I)
                n_susceptible = np.sum(group_S)
                if n_infectious == 0 or n_susceptible == 0:
                    continue
                inf_density = n_infectious / max(len(students_in_loc)-1, 1)
                prob_no_inf = (1 - (inf_density * disease.transmission_rate)) ** contact_rates[loc_idx]
                prob_inf = 1 - prob_no_inf
                rand_vals = np.random.rand(len(students_in_loc))
                newly_infected = group_S & (rand_vals < prob_inf)
                # Log infection events
                infectious_indices = np.array(students_in_loc)[group_I]
                susceptible_indices = np.array(students_in_loc)[newly_infected]
                for target in susceptible_indices:
                    if len(infectious_indices) > 0:
                        source = np.random.choice(infectious_indices)
                    else:
                        source = None
                    infection_events.append({
                        'time': t,
                        'disease': disease.name,
                        'source_student': source,
                        'target_student': target,
                        'location': loc
                    })
                # Update S/I for these location members
                S_next[students_in_loc, d] = S_d[students_in_loc] & ~newly_infected
                I_next[students_in_loc, d] = (I_d[students_in_loc] | newly_infected)
                # Recovery
                rand_vals_recovery = np.random.rand(len(students_in_loc))
                newly_recovered = I_d[students_in_loc] & (rand_vals_recovery < disease.recovery_rate)
                R_next[students_in_loc, d] = R[students_in_loc, d] | newly_recovered
                I_next[students_in_loc, d] = I_next[students_in_loc, d] & ~newly_recovered

    # --- Finalize and assign output ---
    current_states = df[disease_col].to_numpy()
    next_states = np.array(_convert_matrices_to_state(S_next, I_next, R_next))
    final_states = np.where(notna_mask, next_states, current_states)
    df[next_disease_col] = final_states

    # --- Update infection log DataFrame ---
    if infection_log is not None:
        infection_log = pd.concat([infection_log, pd.DataFrame(infection_events)], ignore_index=True)
    else:
        infection_log = pd.DataFrame(infection_events)

    return df, infection_log

In [8]:
# Run SIR model for 48 hours after T=12 (i.e., T=12 to T=60)
# Ensure all Disease_T_* columns are string type
for col in df.columns:
    if col.startswith('Disease_T_'):
        df[col] = df[col].astype(str)

# Run SIR model for 48 hours after T=12 (i.e., T=12 to T=60) with progress bar
infection_log = pd.DataFrame()
for t in tqdm(range(12, 671), desc="SIR Simulation Progress"):
    df, infection_log = run_sir_step_by_location(df, diseases, t, loc_pop, infection_log)

infection_log.to_csv('infection_events_log.csv', index=False)


# Save the results
df.to_csv('Disease_track_completeV2.csv', index=False)
print("Saved disease progression for 4 weeks (-12 hours) after T=12 to Disease_track_completeV2.csv")

SIR Simulation Progress:   0%|          | 0/659 [00:00<?, ?it/s]

Saved disease progression for 4 weeks (-12 hours) after T=12 to Disease_track_completeV2.csv
